In [ ]:
%load_ext line_profiler
%time import nastranpy

Import model
------------

In [ ]:
model = nastranpy.Model()
%time model.read(['/Users/Alvaro/Desktop/nastran_model_input/nastran_launcher.dat'])
#%lprun -f nastranpy.bdf.cards.card.Card._process_fields model.read(['/Users/Alvaro/Desktop/nastran_model_input/nastran_launcher.dat'])

model.path = '/Users/Alvaro/Desktop/nastran_model_output'

## View log

In [ ]:
%less $model.log_path

## Export model

In [ ]:
model.write([include for include in model.includes if include != 'nastran_launcher.dat'])
#model.write()

Get model info
--------------

In [ ]:
model.info()

Write a model summary to a csv file
-----------------------------------

In [ ]:
model.print_summary()

Write card fields to a csv file
-------------------------------

In [ ]:
model.print_cards(model.cards('grid', includes=['BulkData/3C0734_Sp1_Hng_outbd_v04.bdf']))

Get ID info for a given card type
---------------------------------

In [ ]:
print(model.get_id_info('mpc', detailed=False))
print(model.get_id_slot('grid', 1000))

Get shell geometrical info
--------------------------

In [ ]:
shells_info = {shell.id: (shell.area, shell.normal, shell.centroid) for
               shell in model.cards('e2D')}

Renumber cards: by correlation
------------------------------

In [ ]:
correlation = {
    235437: 4703436,
    235438: 4703437,
    235439: 4703438,
    235440: 4703439,
    235441: 4703440,
    235442: 4703441,
    235443: 4703442,
    235444: 4703443,
    235445: 4703444,
    235446: 4703445,
    235447: 4703446,
    235448: 4703447,
    235449: 4703448,
    235450: 4703449,
    235451: 4703450,
    235452: 4703451,
    235453: 4703452,
    235454: 4703453,
    235455: 4703454,
    235456: 4703455,
    235457: 4703456,
    235458: 4703457,
    235459: 4703458,
    235460: 4703459,
    235461: 4703460,
    235462: 4703461,
    235463: 4703462,
    235464: 4703463,
    235465: 4703464,
}

model.renumber('grid', correlation=correlation)
model.write(['BulkData/3C0734_Sp1_Hng_outbd_v04.bdf'])

Renumber cards: by start id and step
------------------------------------

In [ ]:
id_list = [
    235472,
    235473,
    235474,
    235475,
    235476,
    235477,
    235478,
    235479,
    235480,
    235481,
    235482,
    235483,
    235484,
    235485,
    235486,
    235487,
    235488,
    235489,
    235490,
]

model.renumber('grid', model.cards('grid', id_list), start=4703465, step=5)
model.write(['BulkData/3C0734_Sp1_Hng_outbd_v04.bdf'])

Renumber cards: by an id pattern
--------------------------------

In [ ]:
id_list = [
    235496,
    235497,
    235498,
    235499,
    235500,
    235501,
    235502,
    235503,
    235504,
    235505,
    235506,
    235507,
    235508,
    235509,
    235510,
    235511,
    235512,
    235513,
    235514,
    235515,
]

model.renumber('grid', model.cards('grid', id_list), id_pattern=['9', '34', '*', '*', '*', '*', '1-8'])
model.write(['BulkData/3C0734_Sp1_Hng_outbd_v04.bdf'])

Get cards by different ways
---------------------------

In [ ]:
grid_ids = [grid.id for grid in model.cards('grid', ['*', '34', '*', '*', '*', '*', '1-8'])]
print(len(grid_ids))
cards = {card for card in model.cards('grid', grid_ids)}
cards = {card for card in model.cards('e2D')}
cards = {card for card in model.cards('grid')}
cards = {card for card in model.cards(['PCOMP', 'PCOMPG'])}
cards = {card for card in model.cards(includes=['BulkData/3C0734_Sp1_Hng_outbd_v04.bdf', 'BulkData/NEW_LP3.bdf'])}

Extend element: by steps
------------------------

In [ ]:
# Extend from an element
model.elems[3612829].extend(steps=2)
# Extend from a grid
model.grids[3815443].extend(steps=2)

Extend element: by filter
-------------------------

In [ ]:
# Extend from an element
model.elems[8048206].extend('e2D')
# Extend from a grid
model.grids[8020333].extend('e2D')

Make include self-contained
---------------------------

In [ ]:
include = model.includes['BulkData/NEW_MFTE_IFEM_SECTION_V8.5.bdf']
print('Include is self-contained: {}'.format(include.is_self_contained()))
include.make_self_contained()
print('Include is self-contained: {}'.format(include.is_self_contained()))
model.write(['BulkData/NEW_MFTE_IFEM_SECTION_V8.5.bdf'])

In [ ]:
include = model.includes['BulkData/NEW_MFTE_IFEM_SECTION_V8.5.bdf']
print('Include is self-contained: {}'.format(include.is_self_contained()))

In [ ]:
model.coords[3221001].include

Move cards
----------

In [ ]:
print(len(list(model.includes['BulkData/NEW_LP3.bdf'].elems())))
print(len(list(model.includes['BulkData/SMM3v2_S541700_Wing-Box_V16.2_08.bdf'].elems())))
model.move(model.cards('elem', includes=['BulkData/NEW_LP3.bdf']), 'BulkData/SMM3v2_S541700_Wing-Box_V16.2_08.bdf',True)
print(len(list(model.includes['BulkData/NEW_LP3.bdf'].elems())))
print(len(list(model.includes['BulkData/SMM3v2_S541700_Wing-Box_V16.2_08.bdf'].elems())))

## Delete unused cards

In [ ]:
print(model.get_unused_cards('mat'))

In [ ]:
resolved_cards = set()
unresolved_cards = set(model.cards_by_type([Item.grid, Item.coord]))
cards2resolve = set()
for card in unresolved_cards:

    if all((linked_card in resolved_cards for linked_card in card.cards())):
        cards2resolve.add(card)

In [ ]:
# card = model.coords[98000182]
card = model.elems[16300164]
# card = model.props[3311059]
model.delete_card(card)
print(card)
print(list(card.dependent_cards()))

# Get skin bay properties

In [ ]:
bdf_files = [
    '/Users/Alvaro/Desktop/skin_bays/ZA10_Fuselage_itera7_20180704.bdf'
]
EIDs_file = '/Users/Alvaro/Desktop/skin_bays/skin_EIDs.csv'
output_file = '/Users/Alvaro/Desktop/skin_bays/skin_bay_geom.csv'

nastranpy.utils.get_skin_bays_geom(bdf_files, EIDs_file, output_file)